<a href="https://colab.research.google.com/github/babi2707/Trabalho-PAI---Reconhecimento-Papanicolau/blob/etapa2/algoritmo/backend/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
# Célula 1:
import zipfile

caminho_pasta_compactada = '/content/sub_images.zip'
caminho_destino = '/content/sub_images'

with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
    zip_ref.extractall(caminho_destino)

In [123]:
# Célula 2:
import os

base_dir = '/content/sub_images/sub_images'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Criar diretórios principais
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Criar subdiretórios para cada classe
classes = ['ASC-H', 'ASC-US', 'HSIL', 'LSIL', 'Negative for intraepithelial lesion', 'SCC']
for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

In [124]:
# # Célula 3.1:
# from sklearn.model_selection import train_test_split
# import os

# # Diretório onde estão os dados
# data_dir = '/content/sub_images/sub_images'

# # Lista para armazenar os nomes dos arquivos e suas respectivas classes
# file_names = []
# labels = []

# # Preencher as listas com os nomes dos arquivos e suas classes correspondentes
# for class_name in os.listdir(data_dir):
#     class_dir = os.path.join(data_dir, class_name)
#     for file_name in os.listdir(class_dir):
#         file_path = os.path.join(class_dir, file_name)
#         file_names.append(file_path)
#         labels.append(class_name)

# # Dividir os dados em treino e validação (80% treino, 20% validação)
# train_files, val_files, train_labels, val_labels = train_test_split(
#     file_names, labels, test_size=0.2, stratify=labels, random_state=42
# )

# # Dividir os dados de treino em treino e teste (80% treino, 20% teste)
# train_files, test_files, train_labels, test_labels = train_test_split(
#     train_files, train_labels, test_size=0.2, stratify=train_labels, random_state=42
# )

# # Verificar o balanceamento das classes nos conjuntos de treino e teste
# for dataset, dataset_name in zip([train_labels, test_labels], ['treino', 'teste']):
#     unique, counts = np.unique(dataset, return_counts=True)
#     print(f'Balanceamento das classes no conjunto de {dataset_name}:')
#     for class_name, count in zip(unique, counts):
#         print(f'Classe {class_name}: {count} amostras')

# # Agora você tem:
# # - train_files: lista de arquivos de treino
# # - train_labels: lista de rótulos correspondentes de treino
# # - val_files: lista de arquivos de validação
# # - val_labels: lista de rótulos correspondentes de validação
# # - test_files: lista de arquivos de teste
# # - test_labels: lista de rótulos correspondentes de teste

# #Resultados
# # Found 966 images belonging to 6 classes.
# # Found 685 images belonging to 6 classes.
# # Found 704 images belonging to 6 classes.
# # Epoch 1/10
# # 31/31 [==============================] - 38s 158ms/step - loss: 2.9636 - accuracy: 0.2122 - val_loss: 1.7655 - val_accuracy: 0.2015
# # Epoch 2/10
# # 31/31 [==============================] - 3s 103ms/step - loss: 2.4541 - accuracy: 0.2402 - val_loss: 1.7478 - val_accuracy: 0.1985
# # Epoch 3/10
# # 31/31 [==============================] - 5s 156ms/step - loss: 2.2608 - accuracy: 0.2474 - val_loss: 1.7609 - val_accuracy: 0.1985
# # Epoch 4/10
# # 31/31 [==============================] - 3s 102ms/step - loss: 2.0217 - accuracy: 0.2826 - val_loss: 1.8062 - val_accuracy: 0.1985
# # Epoch 5/10
# # 31/31 [==============================] - 3s 100ms/step - loss: 1.9638 - accuracy: 0.2681 - val_loss: 1.7999 - val_accuracy: 0.1985
# # Epoch 6/10
# # 31/31 [==============================] - 3s 110ms/step - loss: 1.8415 - accuracy: 0.2867 - val_loss: 1.7896 - val_accuracy: 0.2234
# # Epoch 7/10
# # 31/31 [==============================] - 4s 143ms/step - loss: 1.8139 - accuracy: 0.2681 - val_loss: 1.7969 - val_accuracy: 0.2146
# # Epoch 8/10
# # 31/31 [==============================] - 3s 102ms/step - loss: 1.7435 - accuracy: 0.3033 - val_loss: 1.8410 - val_accuracy: 0.2219
# # Epoch 9/10
# # 31/31 [==============================] - 3s 105ms/step - loss: 1.7050 - accuracy: 0.2878 - val_loss: 1.8719 - val_accuracy: 0.2161
# # Epoch 10/10
# # 31/31 [==============================] - 5s 156ms/step - loss: 1.6880 - accuracy: 0.3095 - val_loss: 1.8447 - val_accuracy: 0.2263
# # 31/31 [==============================] - 2s 51ms/step
# # 22/22 [==============================] - 1s 49ms/step
# # Validation Accuracy (SVM): 0.19416058394160585
# #                                      precision    recall  f1-score   support

# #                               ASC-H       0.11      0.08      0.09        60
# #                              ASC-US       0.24      0.32      0.27       138
# #                                HSIL       0.18      0.18      0.18       147
# #                                LSIL       0.16      0.24      0.19       136
# # Negative for intraepithelial lesion       0.25      0.16      0.19       144
# #                                 SCC       0.16      0.05      0.08        60

# #                            accuracy                           0.19       685
# #                           macro avg       0.18      0.17      0.17       685
# #                        weighted avg       0.19      0.19      0.19       685

In [125]:
# Célula 3.2:
from sklearn.model_selection import train_test_split
import os

# Diretório onde estão os dados
data_dir = '/content/sub_images/sub_images'

# Lista para armazenar os nomes dos arquivos e suas respectivas classes
file_names = []
labels = []

# Preencher as listas com os nomes dos arquivos e suas classes correspondentes
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        file_names.append(file_path)
        labels.append(class_name)

# Dividir os dados em treino e teste mantendo a proporção de 4:1
train_files = []
test_files = []
train_labels = []
test_labels = []

# Para cada classe, dividir os arquivos em treino e teste mantendo a proporção de 4:1
for class_name in set(labels):
    # Obter os arquivos e rótulos da classe atual
    class_files = [file for file, label in zip(file_names, labels) if label == class_name]
    class_labels = [label for label in labels if label == class_name]

    # Dividir os arquivos e rótulos em treino e teste
    class_train_files, class_test_files, class_train_labels, class_test_labels = train_test_split(
        class_files, class_labels, test_size=0.2, random_state=42)

    # Adicionar os arquivos e rótulos aos conjuntos de treino e teste
    train_files.extend(class_train_files)
    test_files.extend(class_test_files)
    train_labels.extend(class_train_labels)
    test_labels.extend(class_test_labels)

# Verificar o balanceamento das classes nos conjuntos de treino e teste
for dataset, dataset_name in zip([train_labels, test_labels], ['treino', 'teste']):
    unique, counts = np.unique(dataset, return_counts=True)
    print(f'Balanceamento das classes no conjunto de {dataset_name}:')
    for class_name, count in zip(unique, counts):
        print(f'Classe {class_name}: {count} amostras')

# Agora você tem:
# - train_files: lista de arquivos de treino
# - train_labels: lista de rótulos correspondentes de treino
# - test_files: lista de arquivos de teste
# - test_labels: lista de rótulos correspondentes de teste


#Resultados:
# Found 966 images belonging to 6 classes.
# Found 728 images belonging to 6 classes.
# Found 748 images belonging to 6 classes.
# Epoch 1/10
# 31/31 [==============================] - 35s 156ms/step - loss: 2.9200 - accuracy: 0.2112 - val_loss: 1.7805 - val_accuracy: 0.2102
# Epoch 2/10
# 31/31 [==============================] - 3s 110ms/step - loss: 2.4740 - accuracy: 0.2484 - val_loss: 1.7786 - val_accuracy: 0.2102
# Epoch 3/10
# 31/31 [==============================] - 3s 100ms/step - loss: 2.3013 - accuracy: 0.2516 - val_loss: 1.8061 - val_accuracy: 0.2102
# Epoch 4/10
# 31/31 [==============================] - 3s 99ms/step - loss: 2.0867 - accuracy: 0.2640 - val_loss: 1.8524 - val_accuracy: 0.2102
# Epoch 5/10
# 31/31 [==============================] - 5s 168ms/step - loss: 1.9435 - accuracy: 0.2774 - val_loss: 1.8474 - val_accuracy: 0.1992
# Epoch 6/10
# 31/31 [==============================] - 3s 100ms/step - loss: 1.8347 - accuracy: 0.2950 - val_loss: 1.8746 - val_accuracy: 0.2060
# Epoch 7/10
# 31/31 [==============================] - 3s 101ms/step - loss: 1.7898 - accuracy: 0.2743 - val_loss: 1.9190 - val_accuracy: 0.2074
# Epoch 8/10
# 31/31 [==============================] - 3s 106ms/step - loss: 1.7538 - accuracy: 0.3002 - val_loss: 1.9380 - val_accuracy: 0.2102
# Epoch 9/10
# 31/31 [==============================] - 3s 101ms/step - loss: 1.7069 - accuracy: 0.3126 - val_loss: 1.9870 - val_accuracy: 0.2033
# Epoch 10/10
# 31/31 [==============================] - 3s 98ms/step - loss: 1.6868 - accuracy: 0.2940 - val_loss: 1.9681 - val_accuracy: 0.2060
# 31/31 [==============================] - 3s 51ms/step
# 23/23 [==============================] - 1s 48ms/step
# Validation Accuracy (SVM): 0.2087912087912088
#                                      precision    recall  f1-score   support

#                               ASC-H       0.11      0.06      0.08        63
#                              ASC-US       0.20      0.26      0.23       145
#                                HSIL       0.21      0.26      0.23       158
#                                LSIL       0.23      0.23      0.23       146
# Negative for intraepithelial lesion       0.21      0.22      0.22       153
#                                 SCC       0.22      0.03      0.06        63

#                            accuracy                           0.21       728
#                           macro avg       0.20      0.18      0.17       728
#                        weighted avg       0.21      0.21      0.20       728


Balanceamento das classes no conjunto de treino:
Classe ASC-H: 67 amostras
Classe ASC-US: 160 amostras
Classe HSIL: 160 amostras
Classe LSIL: 160 amostras
Classe Negative for intraepithelial lesion: 160 amostras
Classe SCC: 65 amostras
Classe test: 4 amostras
Classe train: 4 amostras
Classe val: 4 amostras
Balanceamento das classes no conjunto de teste:
Classe ASC-H: 17 amostras
Classe ASC-US: 40 amostras
Classe HSIL: 40 amostras
Classe LSIL: 40 amostras
Classe Negative for intraepithelial lesion: 40 amostras
Classe SCC: 17 amostras
Classe test: 2 amostras
Classe train: 2 amostras
Classe val: 2 amostras


In [126]:
# Célula 4:
import shutil
import random

def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.7, 0.15, 0.15)):
    for class_name in classes:
        class_dir = os.path.join(source_dir, class_name)
        images = os.listdir(class_dir)
        random.shuffle(images)

        train_split = int(split_ratio[0] * len(images))
        val_split = int((split_ratio[0] + split_ratio[1]) * len(images))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        for image in train_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(train_dir, class_name, image))

        for image in val_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(val_dir, class_name, image))

        for image in test_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(test_dir, class_name, image))

split_data(base_dir, train_dir, val_dir, test_dir)

In [127]:
# Célula 5:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Definir os caminhos para os dados de treinamento e validação
train_dir = '/content/sub_images/sub_images/train'
val_dir = '/content/sub_images/sub_images/val'
test_dir = '/content/sub_images/sub_images/test'

# Parâmetros
img_height, img_width = 32, 32
batch_size = 32
num_classes = 6
class_names = ['ASC-H', 'ASC-US', 'HSIL', 'LSIL', 'Negative for intraepithelial lesion', 'SCC']

# Criar um gerador de dados para carregar e pré-processar as imagens
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Carregar os dados de treinamento
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Carregar os dados de validação
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Carregar os dados de teste
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
# Carregar a ResNet50 pré-treinada
base_model = ResNet50(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# Adicionar camadas ao modelo
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adicionando dropout
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Ajustando a taxa de aprendizado
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10
)

# Extrair características para treinamento e validação
train_features = model.predict(train_generator)
val_features = model.predict(val_generator)

# Converter rótulos para uma única classe (não one-hot encoding)
train_labels = train_generator.classes
val_labels = val_generator.classes

# Treinar o classificador SVM com hiperparâmetros ajustados
svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=100, gamma=0.1, probability=True))  # Ajustando os hiperparâmetros
svm.fit(train_features, train_labels)

# Avaliar o classificador SVM
val_predictions = svm.predict(val_features)
val_accuracy = accuracy_score(val_labels, val_predictions)
print('Validation Accuracy (SVM):', val_accuracy)
print(classification_report(val_labels, val_predictions, target_names=class_names))

Found 966 images belonging to 6 classes.
Found 792 images belonging to 6 classes.
Found 808 images belonging to 6 classes.
Epoch 1/10
31/31 [==============================] - 37s 225ms/step - loss: 2.9951 - accuracy: 0.1977 - val_loss: 1.7528 - val_accuracy: 0.1907
Epoch 2/10
31/31 [==============================] - 3s 108ms/step - loss: 2.5198 - accuracy: 0.2164 - val_loss: 1.7666 - val_accuracy: 0.2045
Epoch 3/10
31/31 [==============================] - 5s 149ms/step - loss: 2.2088 - accuracy: 0.2526 - val_loss: 1.8285 - val_accuracy: 0.1982
Epoch 4/10
31/31 [==============================] - 4s 116ms/step - loss: 2.0450 - accuracy: 0.2433 - val_loss: 1.8432 - val_accuracy: 0.2033
Epoch 5/10
31/31 [==============================] - 3s 109ms/step - loss: 1.9905 - accuracy: 0.2433 - val_loss: 1.8167 - val_accuracy: 0.2045
Epoch 6/10
31/31 [==============================] - 5s 148ms/step - loss: 1.8529 - accuracy: 0.3002 - val_loss: 1.8809 - val_accuracy: 0.2045
Epoch 7/10
31/31 [======